DESCOBERTAS:
- VIP importa
    * ~50% dos Não-VIPs são transportados
    * ~38% dos VIPs são transportados
    * Terra implica Não-VIP
    * Mas está baixo na lista de prioridade para ser transportado
        * Europa -> ~65% Transportados
        * Earth -> ~41% Transportados -> é o HomePlanet com maior número de pessoas (3566) e todas são NÃO-VIPs
        * Mars -> ~53% Transportados
- Usando cabines para os planetas
    * A B C T -> Europa
    * G -> Earth
- Cryosleep True ou age <= 12 implica nenhum gasto
- Qualquer gasto implica em Cryosleep False
- Msm grupo -> Msm Planeta
- Msm sobrenome -> Msm Planeta

### TESTAR
    
    

In [1]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from collections import defaultdict


pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv("train.csv")
train_df = pd.read_csv("train.csv").dropna()


def getGroup(index):
    return df['PassengerId'].split('_')[0]
#formatting data TODO ISSO VEM POR ULTIMO
df['Transported'] = df['Transported'].map({False: 0, True: 1})
df['Earth'] = np.select([df['HomePlanet'] == 'Earth'], [1], default = 0)
df['Europa'] = np.select([df['HomePlanet'] == 'Europa'], [1], default = 0)
df['Mars'] = np.select([df['HomePlanet'] == 'Mars'], [1], default = 0)
df['CryoSleep'] = np.select([df['CryoSleep'] == True], [1], default = 0)
df['VIP'] = np.select([df['VIP'] == True], [1], default = 0)
df['TRAP'] = np.select([df['Destination'] == 'TRAPPIST-1e'], [1], default = 0)
df['PSO'] = np.select([df['Destination'] == 'PSO J318.5-22'], [1], default = 0)
df['Cancri'] = np.select([df['Destination'] == '55 Cancri e'], [1], default = 0)
groups = []
for i in range(len(df)):
    groups.append((df['PassengerId'][i]).split('_')[0])
df['Groups'] = groups
df['Groups']

0       0001
1       0002
2       0003
3       0003
4       0004
        ... 
8688    9276
8689    9278
8690    9279
8691    9280
8692    9280
Name: Groups, Length: 8693, dtype: object

In [4]:
# This function helps us prove that everybody with the same last name has the same Home Planet.
# The same applies to passenger groups.
def buildHomePlanetDict(column, char, pos, dff):
    '''Function that builds a dictionary that follows a <column, set<planets>> relation'''
    dictionary = {}
    for i in range(len(dff)):
        try:
            current = dff[column][i].split(char)[pos]
            if current not in dictionary:
                dictionary[current] = set()
            if pd.isnull(dff['HomePlanet'][i]) == False:
                dictionary[current].add(dff['HomePlanet'][i])
        except AttributeError:
            continue
    return dictionary

groupDict = buildHomePlanetDict('PassengerId', '_', 0, df)
nameDict = buildHomePlanetDict('Name', ' ', 1, df)


In [6]:
# Fill data
print(df['HomePlanet'].isna().sum()) #201 planetas faltando
df['HomePlanet'] = df['HomePlanet'].fillna(df['Groups'].map(groupDict))
#df['HomePlanet'][i] = list(groupDict[getGroup(i)])[0]
print(df['HomePlanet'].isna().sum())
print(df)


0
0
     PassengerId HomePlanet  CryoSleep     Cabin    Destination   Age  VIP  \
0        0001_01     Europa          0     B/0/P    TRAPPIST-1e  39.0    0   
1        0002_01      Earth          0     F/0/S    TRAPPIST-1e  24.0    0   
2        0003_01     Europa          0     A/0/S    TRAPPIST-1e  58.0    1   
3        0003_02     Europa          0     A/0/S    TRAPPIST-1e  33.0    0   
4        0004_01      Earth          0     F/1/S    TRAPPIST-1e  16.0    0   
...          ...        ...        ...       ...            ...   ...  ...   
8688     9276_01     Europa          0    A/98/P    55 Cancri e  41.0    1   
8689     9278_01      Earth          1  G/1499/S  PSO J318.5-22  18.0    0   
8690     9279_01      Earth          0  G/1500/S    TRAPPIST-1e  26.0    0   
8691     9280_01     Europa          0   E/608/S    55 Cancri e  32.0    0   
8692     9280_02     Europa          0   E/608/S    TRAPPIST-1e  44.0    0   

      RoomService  FoodCourt  ShoppingMall  ...  VRDeck    

In [ ]:
X = np.array(df.drop(columns=['Transported'], axis=1))
y = np.array(df['Transported'])
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.4)


In [ ]:
train_pred = cross_val_predict(RandomForestClassifier(random_state = 1), x_train, y_train, cv = 10, n_jobs=-1)
acuracia_vc = round(accuracy_score(y_train, train_pred)*100, 2)
acuracia_vc

In [ ]:
#Submission block
full_data_model = RandomForestClassifier(random_state = 1)
full_data_model.fit(X,y) #using entire train dataset
test_X = pd.read_csv('test.csv')
# TODO Format test_X



# End TODO
test_preds = full_data_model.predict(test_X)
output = pd.DataFrame({'PassengerId': test_X['PassengerId'], 'Transported': test_preds})
output.to_csv('submission.csv', index=False)

C:\Users\Rafael\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Earth'